In [ ]:
!pip install pyvista
!python -m pip install petsc petsc4py
!pip install numpy
!pip install ufl
#Build Dolfinx
!pip install git+https://github.com/FEniCS/dolfinx.git


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 44.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 23.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done


In [ ]:
from mpi4py import MPI
from petsc4py import PETSc
import numpy as np
import pyvista as pv
from pyvista import examples

from dolfinx.fem import Constant, Function, functionspace, assemble_scalar, dirichletbc, form, locate_dofs_geometrical, solve
from dolfinx.fem.petsc import assemble_matrix, assemble_vector, apply_lifting, create_vector, set_bc, LinearProblem, NonlinearProblem
from dolfinx.nls.petsc import NewtonSolver
from dolfinx.io import VTXWriter
from dolfinx.mesh import create_unit_square, CellType
from dolfinx.plot import vtk_mesh
from basix.ufl import element
from ufl import (FacetNormal, Identity, TestFunction, TrialFunction,
                 div, dot, ds, dx, inner, lhs, nabla_grad, rhs, sym, grad)
import numpy as np

import matplotlib.pyplot as plt

In [ ]:
X_POINTS = 50
Y_POINTS = 50
mesh = create_unit_square(MPI.COMM_WORLD, X_POINTS, Y_POINTS, CellType.quadrilateral)

T = 1
STEPS = 500
dt = T/STEPS
err=1e-8

**References**
```{bibliography}
:filter: docname in docnames
```

In [ ]:
v_cg2 = element("Lagrange", mesh.topology.cell_name(), 2, shape=(mesh.geometry.dim, ))
heat_func_space = functionspace(mesh, v_cg2)

In [ ]:
u = TrialFunction(heat_func_space)
v_test = TestFunction(heat_func_space)

In [ ]:
#Starter function
def add_func(t):
  return 1

In [ ]:
#Boundry Conditions

def on_boundry(point):
  return np.allclose(point, (0.0, point[1]), rtol=err) or np.allclose(point, (1.0, point[1])) or np.allclose(point, (point[0], 0.0)) or np.allclose(point, (point[0], 1.0))
# Locate degrees of freedoon the boundary
dofs = locate_dofs_geometrical(heat_func_space, on_boundry)
val = Constant(mesh, PETSc.ScalarType((0.0, 0.0)))
bc = dirichletbc(val, dofs, heat_func_space)

"""
dofs1 = locate_dofs_geometrical(v_func_space, on_left_boundary)
val = Constant(mesh, PETSc.ScalarType((1.0, 0.0)))
left_bc = dirichletbc(val, dofs1, v_func_space)

dofs2 = locate_dofs_geometrical(v_func_space, lower_boundry)
val2 = Constant(mesh, PETSc.ScalarType((0.0, 0.0)))
lower_bc = dirichletbc(val2, dofs2, v_func_space)

dofs3 = locate_dofs_geometrical(v_func_space, upper_boundry)
val3 = Constant(mesh, PETSc.ScalarType((0.0, 0.0)))
higher_bc = dirichletbc(val3, dofs3, v_func_space)
velocity_boundry_conditions = [left_bc, lower_bc, higher_bc]
"""

In [ ]:
# Define 3D grid points
x_vals = np.linspace(0, 1, X_POINTS)
y_vals = np.linspace(0, 1, Y_POINTS)

z_vals = np.array(curve_data)
x, y, z = np.meshgrid(x_vals, y_vals, z_vals, indexing='ij')

#Flatten For x,y,z

x_flat = x.ravel()
y_flat = y.ravel()
z_flat = z.ravel()

#F(x, y, z) = (u, v, w) where u,v,w = Combination(x_flat, y_flat, z_flat, +, -, * /)
u = x_flat
v = y_flat
w = z_flat

In [ ]:
## Stack point coordinates
points = np.stack((x_flat, y_flat, z_flat), axis=1)

# Create a PyVista point cloud
point_cloud = pv.PolyData(points)
#for variable vectors
vectors = np.stack((u, v, w), axis=1)
#for const
#vectors = np.tile([1, 1, 0], (len(u), 1))
#point_cloud["vectors"] = vectors  # Attach vector data Use glyphs to create arrows representing the vector field
#arrows = point_cloud.glyph(orient="vectors", scale="vectors", factor=0.05)

In [ ]:
#Prepare mesh for visualization
mesh.topology.create_connectivity(mesh.topology.dim, mesh.topology.dim)

"""
Just gets mesh
"""
cells, cell_types, points = vtk_mesh(mesh, mesh.topology.dim)
for i in range(len(cell_types)):
    cell_types[i]=9
grid = pv.UnstructuredGrid(cells, cell_types, points)

curve_data = []
for x in range(X_POINTS+1):
    for y in range(Y_POINTS+1):
        val1 = (x/(X_POINTS+1))**2
        val2 = (1 - ((x/(X_POINTS+1))))**2
        curve_data.append(np.sqrt(val1 + val2))

grid.point_data["curves"] = np.array(curve_data)
grid.set_active_scalars("curves")

warped = grid.warp_by_scalar("curves")

In [ ]:
# Plot
pv.start_xvfb()  # Starts a virtual framebuffer if needed
plotter = pv.Plotter(off_screen=True)
plotter.open_gif("heat_manifold.gif")

#time steps loop
u_next = Function(heat_func_space)
u_n = Function(heat_func_space)
for t in range(STEPS):
  u_new = Function(heat_func_space)
  lhs = (1/dt) * u * v_test * dx + dot(grad(u), grad(v_test)) * dx
  rhs = (1/dt) * u_n * v_test * dx + add_func(t/STEPS) * v_test * dx
  solve(lhs == rhs, u_next, bcs=bc)
  u_n.assign(u_new)

  #Clear Frame
  plotter.clear()
  #Add Mesh to Plotter
  plotter.add_mesh(warped, scalars="u", cmap="coolwarm", clim=[0, 1])
  #Add T =
  plotter.add_text(f"Time: {t/STEPS:.2f}", font_size=10)
  #Save Frame
  plotter.write_frame()
plotter.close()

#added_mesh = plotter.add_mesh(arrows)


In [ ]:
manifold = plotter.add_mesh(warped, edge_color="#000000", color="#FFFFFF" show_edges=True, show_scalar_bar=True)
pic = plotter.screenshot("manifold.png")  # Save to image file
plt.imshow(plotter.image)
plt.show()